# 第七章　給湯設備

## 第一節　給湯設備

1．適用範囲～4．記号及び単位は省略

### 5. 給湯設備によるエネルギー消費量

#### 5.1 消費電力量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{E,W,d} = E_{E,hs,d} + E_{E,aux,ss,d}
$$

$ E_{E,W,d} $：日付$d$における 1 日当たりの給湯設備の消費電力量 (kWh/d)  
$ E_{E,aux,ss,d} $：日付$d$における 1 日当たりの太陽熱利用設備の補機の消費電力量 (kWh/d)    
$ E_{E,hs,d} $：日付$d$における1 日当たりの給湯機（給湯温水暖房機における給湯部も含む。以下、同じ。）の消費電力量 (kWh/d)  

日付$d$における1 日当たりの太陽熱利用設備の補機の消費電力量$ E_{E,aux,ss,d} $は、液体集熱式太陽熱利用設備及び空気集熱式太陽熱利用設備のどちらも採用しない場合、ゼロとする。

#### 5.2 ガス消費量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{G,W,d} = E_{G,hs,d}
$$

$ E_{G,W,d} $：日付$d$における 1 日当たりの給湯設備のガス消費量 (MJ/d)  
$ E_{G,hs,d} $：日付$d$における1 日当たりの給湯機のガス消費量 (MJ/d)  

#### 5.3 灯油消費量

台所、洗面所及び浴室等がいずれも無い場合は0とする。 

$$\large
\displaystyle E_{K,W,d} = E_{K,hs,d}
$$

$ E_{K,W,d} $：日付$d$における 1 日当たりの給湯設備の灯油消費量 (MJ/d)  
$ E_{K,hs,d} $：日付$d$における1 日当たりの給湯機の灯油消費量 (MJ/d)  

#### 5.4 その他の燃料による一次エネルギー消費量

日付$d$における 1 日当たりの給湯設備のその他の燃料による一次エネルギー消費量$ E_{M,w,d} $は0とする。 

### 6. 給湯機のエネルギー消費量

省略

### 7. 太陽熱補正給湯熱負荷

$$\large
\displaystyle
L''_{k,d} = L'_{k,d} - L_{sun,d} \times \frac{L'_{k,d}}{(L'_{k,d}+L'_{s,d}+L'_{w,d}+L'_{b1,d}+L'_{b2,d}+L'_{ba1,d})}
$$


$$\large
\displaystyle L''_{s,d} = L'_{s,d} - L_{sun,d} \times \frac { L'_{s,d}}{ (L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} ) }
$$


$$\large
\displaystyle L''_{w,d} = L'_{w,d} - L_{sun,d} \times \frac { L'_{w,d}}{ (L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} ) }
$$  


$$\large
\displaystyle L''_{b1,d}= L'_{b1,d} - L_{sun,d} \times \frac{ L'_{b1,d}} {(L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} ) }
$$


$$\large
\displaystyle L''_{b2,d}= L'_{b2,d} - L_{sun,d} \times \frac{ L'_{b2,d}} {(L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d} ) }
$$


$$\large
\displaystyle L''_{ba1,d} = L'_{ba1,d} - L_{sun,d} \times \frac{L'_{ba1,d}}{(L'_{k,d} + L'_{s,d} + L'_{w,d} + L'_{b1,d} + L'_{b2,d} + L'_{ba1,d})}
$$

$$\large
\displaystyle L''_{ba2,d} = L'_{ba2,d}
$$

$ L''_{k,d} $：日付$d$における 1 日当たりの台所水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{s,d} $：日付$d$における 1 日当たりの浴室シャワー水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{w,d} $：日付$d$における 1 日当たりの洗面水栓における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{b1,d} $：日付$d$における 1 日当たりの浴槽水栓湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{b2,d} $：日付$d$における 1 日当たりの浴槽自動湯はり時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{ba1,d} $：日付$d$における 1 日当たりの浴槽水栓さし湯時における太陽熱補正給湯熱負荷(MJ/d)  
$ L''_{ba2,d} $：日付$d$における 1 日当たりの浴槽追焚時における太陽熱補正給湯熱負荷(MJ/d)  
$ L'_{k,d} $：日付$d$における 1 日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d)  
$ L'_{s,d} $：日付$d$における 1 日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d)  
$ L'_{w,d} $：日付$d$における 1 日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d)  
$ L'_{b1,d} $：日付$d$における 1 日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d)  
$ L'_{b2,d} $：日付$d$における 1 日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d)  
$ L'_{ba1,d} $：日付$d$における 1 日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d)  
$ L'_{ba2,d} $：日付$d$における 1 日当たりの浴槽追焚時における節湯補正給湯熱負荷(MJ/d)  
$ L_{sun,d} $：日付$d$における 1 日当たりの太陽熱利用給湯設備による補正集熱量(MJ/d)

日付$d$における 1 日当たりの太陽熱利用給湯設備による補正集熱量$ L_{sun,d} $は、液体集熱式太陽熱利用設備及び空気集熱式太陽熱利用設備のどちらも採用しない場合、ゼロとする。

### 8. 節湯補正給湯熱負荷

$$\large
\displaystyle L'_{k,d} = W'_{k,d} \times (\theta_{sw,k} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{s,d} = W'_{s,d} \times (\theta_{sw,s} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{w,d} = W'_{w,d} \times (\theta_{sw,w} - \theta_{wtr,d}) \times 4.186 \times 10^{-3}
$$

浴室等が無い場合（ふろ機能の種類によらない）

$$\large
\displaystyle L'_{b1,d} = 0.0
$$

$$\large
\displaystyle L'_{b2,d} = 0.0
$$

浴室等が有り、ふろ機能の種類が給湯単機能の場合 

$$\large
\displaystyle L'_{b1,d} = W'_{b1,d} \times ( \theta_{sw,b1} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{b2,d} = 0.0
$$

浴室等が有り、ふろ機能の種類がふろ給湯機（追焚の有無によらない）の場合

$$\large
\displaystyle L'_{b1,d} = 0.0
$$

$$\large
\displaystyle L'_{b2,d} = W'_{b2,d} \times ( \theta_{sw,b2} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

浴室等が無い場合

$$\large
\displaystyle L'_{ba1,d} = 0.0
$$

$$\large
\displaystyle L'_{ba2,d} = 0.0
$$

浴室等が有り、ふろ機能の種類が給湯単機能またはふろ給湯機（追焚なし）の場合 

$$\large
\displaystyle L'_{ba1,d} = W'_{ba1,d} \times ( \theta_{sw,ba1} - \theta_{wtr,d} ) \times 4.186 \times 10^{-3}
$$

$$\large
\displaystyle L'_{ba2,d} = 0.0
$$

浴室等が有り、ふろ機能の種類がふろ給湯機（追焚あり）の場合

$$\large
\displaystyle L'_{ba1,d} = 0.0
$$

$$\large
\displaystyle L'_{ba2,d} = L_{ba,d} \times 1.25
$$

$ L'_{k,d} $：日付$d$における 1 日当たりの台所水栓における節湯補正給湯熱負荷(MJ/d)  
$ L'_{s,d} $：日付$d$における 1 日当たりの浴室シャワー水栓における節湯補正給湯熱負荷(MJ/d)  
$ L'_{w,d} $：日付$d$における 1 日当たりの洗面水栓における節湯補正給湯熱負荷(MJ/d)  
$ L'_{b1,d} $：日付$d$における 1 日当たりの浴槽水栓湯はり時における節湯補正給湯熱負荷(MJ/d)  
$ L'_{b2,d} $：日付$d$における 1 日当たりの浴槽自動湯はり時における節湯補正給湯熱負荷(MJ/d)  
$ L'_{ba1,d} $：日付$d$における 1 日当たりの浴槽水栓さし湯時における節湯補正給湯熱負荷(MJ/d)  
$ L'_{ba2,d} $：日付$d$における 1 日当たりの浴槽追焚時における節湯補正給湯熱負荷(MJ/d)  
$ W'_{k,d} $：日付$d$における 1 日当たりの台所水栓における節湯補正給湯量(L/d)  
$ W'_{s,d} $：日付$d$における 1 日当たりの浴室シャワー水栓における節湯補正給湯量(L/d)  
$ W'_{w,d} $：日付$d$における 1 日当たりの洗面水栓における節湯補正給湯量(L/d)  
$ W'_{b1,d} $：日付$d$における 1 日当たりの浴槽水栓湯はり時における節湯補正給湯量(L/d)  
$ W'_{b2,d} $：日付$d$における 1 日当たりの浴槽自動湯はり時における節湯補正給湯量(L/d)  
$ W'_{ba1,d} $：日付$d$における 1 日当たりの浴槽水栓さし湯時における節湯補正給湯量(L/d)  
$ W'_{ba2,d} $：日付$d$における 1 日当たりの浴槽追焚時における節湯補正給湯量(L/d)  
$ L_{ba,d} $：日付$d$における 1 日当たりの浴槽沸かし直しによる給湯熱負荷(MJ/d)  
$ \theta_{sw,k} $：台所水栓における基準給湯温度(℃)  
$ \theta_{sw,s} $：浴室シャワー水栓における基準給湯温度(℃)  
$ \theta_{sw,w} $：洗面水栓における基準給湯温度(℃)  
$ \theta_{sw,b1} $：浴槽水栓湯はり時における基準給湯温度(℃)  
$ \theta_{sw,b2} $：浴槽自動湯はり時における基準給湯温度(℃)  
$ \theta_{sw,ba1} $：浴槽水栓さし湯時における基準給湯温度(℃)  
$ \theta_{sw,ba2} $：浴槽追焚時における基準給湯温度(℃)  
$ \theta_{wtr,d} $：日付$d$における日平均給水温度(℃)

<div style="text-align: center;font-weight: bold;">表　用途ごとの基準給湯温度</div>

| 用途 |　記号　| 基準給湯温度(℃) |
| ----- | ---- | ---- |
| 台所水栓 | $\theta_{sw,k}$ | 40 |
|浴室シャワー水栓|$\theta_{sw,s}$|40|
|洗面水栓|$\theta_{sw_w}$|40|
|浴槽水栓湯はり|$\theta_{sw,b1}$|40|
|浴槽自動湯はり|$\theta_{sw,b1}$|40|
|浴槽水栓さし湯|$\theta_{sw,ba1}$|60|


### 9. 節湯補正給湯量

$$\large
\displaystyle W'_{k,d} = W_{k,d} \times f_{sk} \times f_{sp}
$$

$$\large
\displaystyle W'_{s,d} = W_{s,d} \times f_{ss} \times f_{sp}
$$

$$\large
\displaystyle W'_{w,d} = W_{w,d} \times f_{sw} \times f_{sp}
$$

$$\large
\displaystyle W'_{b1,d} = W_{b1,d} \times f_{sp} \times f_{sb}
$$

$$\large
\displaystyle W'_{b2,d} = W_{b2,d} \times f_{sb}
$$

$$\large
\displaystyle W'_{ba1,d} = W_{ba1,d} \times f_{sp}
$$

$ W'_{k,d} $：日付$d$における 1 日当たりの台所水栓における節湯補正給湯量(L/d)  
$ W'_{s,d} $：日付$d$における 1 日当たりの浴室シャワー水栓における節湯補正給湯量(L/d)  
$ W'_{w,d} $：日付$d$における 1 日当たりの洗面水栓における節湯補正給湯量(L/d)  
$ W'_{b1,d} $：日付$d$における 1 日当たりの浴槽水栓湯はり時における節湯補正給湯量(L/d)  
$ W'_{b2,d} $：日付$d$における 1 日当たりの浴槽自動湯はり時における節湯補正給湯量(L/d)  
$ W'_{ba1,d} $：日付$d$における 1 日当たりの浴槽水栓さし湯時における節湯補正給湯量(L/d)  
$ W'_{ba2,d} $：日付$d$における 1 日当たりの浴槽追焚時における節湯補正給湯量(L/d)  
$ W_{k,d} $：日付$d$における 1 日当たりの台所水栓における基準給湯量(L/d)  
$ W_{s,d} $：日付$d$における 1 日当たりの浴室シャワー水栓における基準給湯量(L/d)  
$ W_{w,d} $：日付$d$における 1 日当たりの洗面水栓における基準給湯量(L/d)  
$ W_{b1,d} $：日付$d$における 1 日当たりの浴槽水栓湯はり時における基準給湯量(L/d)  
$ W_{b2,d} $：日付$d$における 1 日当たりの浴槽自動湯はり時における基準給湯量(L/d)  
$ W_{ba1,d} $：日付$d$における 1 日当たりの浴槽水栓さし湯時における基準給湯量(L/d)  
$ W_{ba2,d} $：日付$d$における 1 日当たりの浴槽追焚時における基準給湯量(L/d)  
$ f_{sk} $：台所水栓における節湯の効果係数  
$ f_{ss} $：浴室シャワー水栓における節湯の効果係数  
$ f_{sw} $：洗面水栓における節湯の効果係数  
$ f_{sp} $：配管における節湯の効果係数  
$ f_{sb} $：浴槽における節湯の効果係数  

### 10. 基準給湯量

$$\large
\displaystyle
W_{k,d} = \left\{ \begin{array}{ll}
    W_{k,p,d}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{k,p,d}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{k,p,d}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{k,p,d}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{k,p,d}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{k,p,d}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$$\large
\displaystyle
W_{s,d} = \left\{ \begin{array}{ll}
    W_{s,p,d}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{s,p,d}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{s,p,d}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{s,p,d}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{s,p,d}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{s,p,d}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$$\large
\displaystyle
W_{w,d} = \left\{ \begin{array}{ll}
    W_{w,p,d}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{w,p,d}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{w,p,d}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{w,p,d}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{w,p,d}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{w,p,d}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$

$ W_{k,d} $：日付$d$における 1 日当たりの台所水栓における基準給湯量(L/d)  
$ W_{k,p,d} $：日付$d$における 1 日当たりの居住人数が$p$人における台所水栓における基準給湯量(L/d)  
$ W_{s,d} $：日付$d$における 1 日当たりの浴室シャワー水栓における基準給湯量(L/d)  
$ W_{s,p,d} $：日付$d$における 1 日当たりの居住人数が$p$人における浴室シャワー水栓における基準給湯量(L/d)  
$ W_{w,d} $：日付$d$における 1 日当たりの洗面水栓における基準給湯量(L/d)  
$ W_{w,p,d} $：日付$d$における 1 日当たりの居住人数が$p$人における洗面水栓における基準給湯量(L/d)  
$ n_{p} $：仮想居住人数

$$\large
\displaystyle
W_{b,d} = \left\{ \begin{array}{ll}
    W_{b,p,d}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + W_{b,p,d}|_{p=2} \displaystyle \times \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    W_{b,p,d}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + W_{b,p,d}|_{p=3} \displaystyle \times \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    W_{b,p,d}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + W_{b,p,d}|_{p=4} \displaystyle \times \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$

$ W_{b,d} $：日付$d$における 1 日当たりの浴槽湯はり時における基準給湯量(L/d)  
$ W_{b,p,d} $：日付$d$における 1 日当たりの居住人数が$p$人における浴槽湯はり時における基準給湯量(L/d)  
$ n_{p} $：仮想居住人数

<div style="text-align: center;font-weight: bold;">表　居住人数 4 人における基準給湯量（L/d）</div>

<table>
<thead>
<tr>
<th rowspan="2">生活スケジュール</th>
<th rowspan="2">台所水栓 $W_{k}$</th>
<th colspan="2"><center>浴室シャワー水栓 $W_{s}$</center></th>
<th rowspan="2">洗面水栓 $W_{w}$</th>
<th colspan="2"><center>浴槽湯はり $W_{b}$</center></th>
</tr>
<tr>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
</tr>
</thead>
    
<tbody>
<tr><td>休日在宅（大）</td><td>184</td><td>240</td><td>0</td><td>46</td><td> 180</td><td> 0</td></tr>
<tr><td>休日在宅（小）</td><td>140</td><td> 200</td><td> 0</td><td>30</td><td> 180</td><td> 0</td></tr>
<tr><td>平日（大）</td><td>136</td><td> 200</td><td> 0</td><td>34</td><td> 180</td><td> 0</td></tr>
<tr><td>平日（中）</td><td>92</td><td> 150</td><td> 0</td><td>28</td><td> 180</td><td> 0</td></tr>
<tr><td>平日（小）</td><td>62</td><td> 100</td><td> 0</td><td> 8</td><td> 180</td><td> 0</td></tr>
<tr><td>休日外出</td><td>32</td><td> 190</td><td> 0</td><td>28</td><td> 0</td><td> 0</td></tr>
</tbody>
</table>


<div style="text-align: center;font-weight: bold;">表　居住人数 3 人における基準給湯量（L/d）</div>

<table>
<thead>
<tr>
<th rowspan="2">生活スケジュール</th>
<th rowspan="2">台所水栓 $W_{k}$</th>
<th colspan="2"><center>浴室シャワー水栓 $W_{s}$</center></th>
<th rowspan="2">洗面水栓 $W_{w}$</th>
<th colspan="2"><center>浴槽湯はり $W_{b}$</center></th>
</tr>
<tr>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
</tr>
</thead>
    
<tbody>
<tr><td>休日在宅（大）</td><td>164</td><td>210</td><td>0</td><td>46</td><td>180</td><td>0</td></tr>
<tr><td>休日在宅（小）</td><td>136</td><td>160</td><td>0</td><td>24</td><td>180</td><td>0</td></tr>
<tr><td>平日（大）</td><td>132</td><td>160</td><td>0</td><td>28</td><td>180</td><td>0</td></tr>
<tr><td>平日（中）</td><td>88</td><td>110</td><td>0</td><td>22</td><td>180</td><td>0</td></tr>
<tr><td>平日（小）</td><td>48</td><td>60</td><td>0</td><td>12</td><td>180</td><td>0</td></tr>
<tr><td>休日外出</td><td>58</td><td>110</td><td>0</td><td>32</td><td>0</td><td>0</td></tr>
</tbody>
</table>


<div style="text-align: center;font-weight: bold;">表　居住人数 2 人における基準給湯量（L/d）</div>

<table>
<thead>
<tr>
<th rowspan="2">生活スケジュール</th>
<th rowspan="2">台所水栓 $W_{k}$</th>
<th colspan="2"><center>浴室シャワー水栓 $W_{s}$</center></th>
<th rowspan="2">洗面水栓 $W_{w}$</th>
<th colspan="2"><center>浴槽湯はり $W_{b}$</center></th>
</tr>
<tr>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
</tr>
</thead>
    
<tbody>
<tr><td>休日在宅（大）</td><td>128</td><td>130</td><td>0</td><td>32</td><td>180</td><td>0</td></tr>
<tr><td>休日在宅（小）</td><td>96</td><td>70</td><td>0</td><td>24</td><td>180</td><td>0</td></tr>
<tr><td>平日（大）</td><td>96</td><td>70</td><td>0</td><td>24</td><td>180</td><td>0</td></tr>
<tr><td>平日（中）</td><td>44</td><td>30</td><td>0</td><td>16</td><td>180</td><td>0</td></tr>
<tr><td>平日（小）</td><td>44</td><td>110</td><td>0</td><td>16</td><td>0</td><td>0</td></tr>
<tr><td>休日外出</td><td>38</td><td>20</td><td>0</td><td>12</td><td>0</td><td>0</td></tr>
</tbody>
</table>


<div style="text-align: center;font-weight: bold;">表　居住人数 1 人における基準給湯量（L/d）</div>

<table>
<thead>
<tr>
<th rowspan="2">生活スケジュール</th>
<th rowspan="2">台所水栓 $W_{k}$</th>
<th colspan="2"><center>浴室シャワー水栓 $W_{s}$</center></th>
<th rowspan="2">洗面水栓 $W_{w}$</th>
<th colspan="2"><center>浴槽湯はり $W_{b}$</center></th>
</tr>
<tr>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
<th>浴室等がある場合</th>
<th>浴室等がない場合</th>
</tr>
</thead>
    
<tbody>
<tr><td>休日在宅（大）</td><td>28</td><td>80</td><td>0</td><td>32</td><td>180</td><td>0</td></tr>
<tr><td>休日在宅（小）</td><td>26</td><td>20</td><td>0</td><td>24</td><td>180</td><td>0</td></tr>
<tr><td>平日（大）</td><td>26</td><td>20</td><td>0</td><td>24</td><td>180</td><td>0</td></tr>
<tr><td>平日（中）</td><td>54</td><td>90</td><td>0</td><td>36</td><td>0</td><td>0</td></tr>
<tr><td>平日（小）</td><td>34</td><td>60</td><td>0</td><td>16</td><td>0</td><td>0</td></tr>
<tr><td>休日外出</td><td>8</td><td>20</td><td>0</td><td>12</td><td>0</td><td>0</td></tr>
</tbody>
</table>


日付$d$における 1 日当たりの浴槽水栓さし湯時における基準給湯量$ W_{ba1,d} $は、ふろ機能の種類が給湯単機能又はふろ給湯機（追焚なし）の場合のみに発生する。

$$\large
\displaystyle W_{ba1,d} = L_{ba,d} \times \frac{1}{(\theta_{sw,ba1} - \theta_{wtr,d})} \times \frac{1}{4.186} \times 10^3
$$

$ W_{ba1,d} $：日付$d$における1 日当たりの浴槽水栓さし湯時における基準給湯量(L/d)  
$ L_{ba,d} $：日付$d$における1 日当たりの浴槽沸かし直しによる給湯熱負荷(MJ/d)  
$ \theta_{sw,ba1} $：浴槽水栓さし湯時における基準給湯温度(℃)  
$ \theta_{wtr,d} $：日付$d$における日平均給水温度(℃)  

### 11. 浴槽沸かし直しによる給湯熱負荷

$$\large
\displaystyle
L_{ba,d} = \left\{ \begin{array}{ll}
    L_{ba,p,d}|_{p=1} \times \displaystyle \frac{2 - n_p}{2 - 1} + L_{ba,p,d}|_{p=2} \times \displaystyle \frac{n_p - 1}{2 - 1} & (1 \leq n_p < 2)\\ 
    L_{ba,p,d}|_{p=2} \times \displaystyle \frac{3 - n_p}{3 - 2} + L_{ba,p,d}|_{p=3} \times \displaystyle \frac{n_p - 2}{3 - 2} & (2 \leq n_p < 3)\\ 
    L_{ba,p,d}|_{p=3} \times \displaystyle \frac{4 - n_p}{4 - 3} + L_{ba,p,d}|_{p=4} \times \displaystyle \frac{n_p - 3}{4 - 3} & (3 \leq n_p \leq 4)\\ 
    \end{array}
\right.
$$


$ L_{ba,d} $：日付$d$における1 日当たりの浴槽沸かし直しによる給湯熱負荷(MJ/d)  
$ L_{ba,p,d} $：日付$d$における1 日当たりの居住人数が$p$人における浴槽沸かし直しによる給湯熱負荷(MJ/d)  
$ n_{p} $：仮想居住人数

$$\large
\displaystyle
L_{ba,p,d} = \left\{ \begin{array}{ll}
    a_{ba,p,d} \times \theta_{ex,Ave,d} + b_{ba,p,d} & (W_{b,p,d} > 0)\\
    0 & (W_{b,p,d} = 0)\\
    \end{array}
\right.
$$

$ L_{ba,p,d} $：日付$d$における1 日当たりの居住人数が$p$人における浴槽沸かし直しによる給湯熱負荷(MJ/d)  
$ a_{ba,p,d}, b_{ba,p,d} $：係数  
$\theta_{ex,Ave,d}$：日付$d$における日平均外気温度(℃)

<div style="text-align: center;font-weight: bold;">表　係数 $a_{ba}$ 及び $b_{ba}$ の値</div>

<table>
<thead>
<tr>
<th rowspan="2">日平均外気温度の条件</th>
<th rowspan="2">浴槽の種類</th>
<th colspan="2"><center>4 人</center></th>
<th colspan="2"><center>3 人</center></th>
<th colspan="2"><center>2 人</center></th>
<th colspan="2"><center>1 人</center></th>
</tr>
<tr>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
<th>係数 $a_{ba}$</th>
<th>係数 $b_{ba}$</th>
</tr>
</thead>

<tbody>
<tr><td rowspan="2">7℃未満</td><td>通常浴槽</td><td>-0.12</td><td>6.00</td><td>-0.10</td><td>4.91</td><td>-0.06</td><td>3.02</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.07</td><td>3.98</td><td>-0.06</td><td>3.22</td><td>-0.04</td><td>2.01</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">7℃以上かつ16℃未満</td><td>通常浴槽</td><td>-0.13</td><td>6.04</td><td>-0.10</td><td>4.93</td><td>-0.06</td><td>3.04</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.08</td><td>4.02</td><td>-0.06</td><td>3.25</td><td>-0.04</td><td>2.03</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">16℃以上かつ25℃未満</td><td>通常浴槽</td><td>-0.14</td><td>6.21</td><td>-0.11</td><td>5.07</td><td>-0.07</td><td>3.13</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.09</td><td>4.19</td><td>-0.07</td><td>3.39</td><td>-0.04</td><td>2.12</td><td>0.00</td><td>0.00</td></tr>
<tr><td rowspan="2">25℃以上</td><td>通常浴槽</td><td>-0.12</td><td>5.81</td><td>-0.10</td><td>4.77</td><td>-0.06</td><td>2.92</td><td>0.00</td><td>0.00</td></tr>
<tr><td>高断熱浴槽</td><td>-0.07</td><td>3.80</td><td>-0.06</td><td>3.09</td><td>-0.04</td><td>1.92</td><td>0.00</td><td>0.00</td></tr>
</tbody>

</table>


### 12. 日平均給水温度

$$\large 
\theta_{wtr,d} = max(a_{wtr} \times \theta_{ex,prd,Ave,d} + b_{wtr}, 0.5)
$$

$ \theta_{wtr,d}$：日付$d$における日平均給水温度(℃)  
$ \theta_{ex,prd,Ave,d}$：日付$d$を基準とした期間平均外気温度(℃)  
$ a_{wtr}, b_{wtr} $：日平均給水温度を求める際の回帰係数  

<div style="text-align: center;font-weight: bold;">表　日平均給水温度を求める際の回帰係数の値</div>

|地域の区分|$a_{wtr}$|$b_{wtr}$|
|-|-|-|
|1|0.6639|3.466|
|2|0.6639|3.466|
|3|0.6054|4.515|
|4|0.6054|4.515|
|5|0.8660|1.665|
|6|0.8516|2.473|
|7|0.9223|2.097|
|8|0.6921|7.167|


$$\large 
\theta_{ex,prd,Ave,d} = \sum^{10}_{k = 1} \theta_{ex,Ave,d} / 10
$$

$ \theta_{ex,Ave,d-k} $：日付$d$を基準とした$k$日前の日平均外気温度(℃)  

### 13. 日平均外気温度

$$\large 
\theta_{ex,Ave,d} = \sum^{23}_{t = 0} \theta_{ex,d,t} / 24
$$

$ \theta_{ex,Ave,d} $：日付$d$における日平均外気温度(℃)  
$ \theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  

### 14. 夜間平均外気温度

$$\large 
\theta_{ex,Nave,d} = \bigg\{ \theta_{ex,d-1,23} + \sum^6_{t=0}\theta_{ex,d,t} \bigg\}/8
$$

$ \theta_{ex,d-1,23} $：日付$d-1$における23 時における外気温度(℃)  
$ \theta_{ex,d,t}$：日付$d$の時刻$t$における外気温度(℃)  

### 15. 温水暖房の熱負荷

$$\large 
L_{HWH,d} = \sum^{23}_{t=0} L_{HWH,d,t}
$$

$ L_{HWH,d} $：日付$d$における1 日当たりの温水暖房の熱負荷(MJ/d)  
$ L_{HWH,d,t}$：日付$d$の時刻$t$における1 時間当たりの温水暖房の熱負荷(MJ/h)  

## 付録 A 複数の給湯機を設置する場合の評価において代表させる給湯機の決定方法 

省略

## 付録 B 給湯機を設置しない場合に評価において想定する機器の種類と仕様 

省略